### ДЗ 5: WSD

#### 1) Код - здесь ниже
* Vectorizers: Count, Tf/idf
* Classifiers: multinomial naive Bayes, Random Forest, Decision Tree

#### 2) Данные:
https://github.com/phuuda/nlp_year4/blob/master/hw5_wsd/castle-lock.txt <br><br>

Данные + 10 собственных примеров:<br>
https://github.com/phuuda/nlp_year4/blob/master/hw5_wsd/test_contexts.txt

#### 3) Описание:

Данные:

<b>а.</b> Source: Sketch Engine, ruTenTen11, lemma searched = 'замок'

<b>б.</b><br>
* 700 контекстов употребления 'замок' = castle -- class 1
* 700 контекстов употребления 'замок' = lock -- class 0
* тексты ruTenTen11 -- из интернета
* значение лексемы в каждом примере размечалось вручную
* контексты -- примено 10-15 слов

<b>в.</b><br>
* лемматизатор -- Mystem
* стоп-лист -- расширенный список на основе Вордстата: http://www.bukvarix.com/bukvarix-stop-words.html<br>

<b>г.</b> примеры ошибок для модели <b>Initial vectors + Random Forest + Tf idf</b>:<br>
* <i>ночью замок был освещен и был открыт для посетителей и ночных экскурсий</i> (неясно, почему ошибочно классифицирует как 'lock'. видимо, не достаточно key-words про средневековье (<i>рыцари</i>, <i>старинный</i>, <i>королевский</i>, etc.))
* <i>ребенок пытался вставить карандаш в скважину и громко орал</i> (тоже не очень ясно, видимо из-за отсутствия слов <i>дверь</i>, <i>ключ</i>, <i>металл</i>, and similar.)

<b>д.</b> таблица с результатами по 6 моделям:<br>
<img src="https://raw.githubusercontent.com/phuuda/nlp_year4/master/hw5_wsd/hw5_wsd_results.png"><br>

* Для initial & reduced vectors Pipeline давал estimation, что <b>naive Bayes</b> + Count Vectorizer выдаст наибольший accuracy (89% и 82% respectively)<br><br>

* Для reduced vectors это частично подтвердилось: <b>Bayes</b> выдавал наибольший accuracy of classification, совпадающий у Tf/idf & Count Vectorizers (85%).<br><br>

* Однако, для initial (non-reduced) vectors <b>Random Forest</b> опережает остальные модели (90% accuracy score).<br><br>

* Интересно, что при inital vectors <b>Random Forest, Decision Tree</b>, и <b>Bayes</b> справляются довольно неплохо (81-90% accuracy), а для reduced vectors <b>Random Forest</b> и <b>Decision</b> резко ухудшают свои результаты (58-73% accuracy). В то же время, при reduced vectors <b>Bayes</b> работает одинаково хорошо при Tf/idf & Count Vectorizers (85% accuracy).<br><br>

* В среднем, Tf/idf дает лучшие результаты для 3 моделей.<br><br>

<b>е.</b> по 5 примеров ваших собственных на каждое значение (не из датасета) - результат применения к ним лучшего метода - <b>Random Forest + Tf/idf + Initial vectors</b><br><br>
```
1  они объездили всю Европу чтобы посетить самые знаменитые средневековые замки
1  ночью замок был освещен и был открыт для посетителей и ночных экскурсий
1  в подвалах и башнях старого замка было жутко холодно темно и страшновато
1  чтобы завоевать замок нужно было штурмовать еще и ближайшие деревни
1  в одной из башен замка освещенное окошко служило маяком слабой надежды
0  я могу взломать любой замок особенно если нужен не ключ а числовая комбинация
0  хозяева установили новые металлические замки с системой сигнализации
0  я не заметил замка на двери и врезался в нее почти с разбега
0  компания предлагает большой выбор замков оптом и с гарантией на четыре года
0  ребенок пытался вставить карандаш в скважину замка и громко орал

```
<b>Результат: 8/10</b> (ошибочная классификация примеров 2 и 10):
```
'ночью замок был освещен и был открыт для посетителей и ночных экскурсий'
'ребенок пытался вставить карандаш в скважину замка и громко орал'
```

In [508]:
import re
from collections import Counter
import os
from pprint import pprint
from pymystem3 import Mystem

In [509]:
m = Mystem()

### Загрузка стоп-слов

In [510]:
def load_stop_words(stop_filename):
	''' загрузить список стоп-слов из файла, одно слово на строке '''
	with open(stop_filename, encoding = 'utf-8') as f:
		stopwords = [w.strip() for w in f.readlines() if w.strip()]
	return set(stopwords)

stopw = "stoplist_russian.txt"
print(load_stop_words(stopw))

{'этому', 'нам', 'будет', 'два', 'моего', 'того', 'вам', 'своем', 'всею', 'она', 'была', 'и', 'которой', 'моею', 'своя', 'моими', 'кому', 'этими', 'одна', 'таким', 'своему', 'было', 'нем', 'себя', 'можешь', 'моей', 'только', 'этим', 'может', 'быть', 'нею', 'кем', 'наша', 'так', 'моëм', 'могла', 'могли', 'ним', 'таких', 'эту', 'нашим', 'своих', 'вас', 'ее', 'одними', 'даже', 'ему', 'с', 'моим', 'тот', 'нас', 'своей', 'что-то', 'эта', 'его', 'мою', 'свои', 'пока', 'едва', 'наши', 'одного', 'всю', 'вами', 'одной', 'вы', 'были', 'будут', 'всеми', 'ей', 'тебя', 'моем', 'которому', 'всего', 'ещë', 'которым', 'ними', 'моему', 'такою', 'нашу', 'каждый', 'на', 'могут', 'тобой', 'уже', 'этом', 'такому', 'наших', 'главный', 'будь', 'свою', 'собою', 'когда', 'которые', 'я', 'мною', 'ней', 'тобою', 'который', 'мой', 'мое', 'той', 'мои', 'своё', 'в', 'могу', 'такое', 'всех', 'есть', 'моя', 'здесь', 'что', 'не', 'по', 'которого', 'него', 'чего', 'будем', 'еë', 'мной', 'себе', 'одно', 'ими', 'самый', 

### Препроцессинг, лемматизация

In [511]:
def preprocessing(raw_text):
    clean_text = re.sub('\W+', ' ', raw_text) # \W = [^a-zA-Z0-9_]
    return clean_text

def lemmatize(input):
    return [lemma.strip() for lemma in m.lemmatize(preprocessing(input.lower())) if lemma.strip()]

text = "с виду воду, которая бьёт  рядом с французским городом Авен"
m.lemmatize(text)

#r_text = "Он еле дождался, пока я доковыряюсь ключом в замке и, не снимая сапожек, прямиком пошёл в свою спальню."
print(preprocessing(text))
print(lemmatize(text))

с виду воду которая бьёт рядом с французским городом Авен
['с', 'вид', 'вода', 'который', 'бить', 'рядом', 'с', 'французский', 'город', 'авен']


In [512]:
def remove_stop_words(lemmas, stopwords):
    return ' '.join([word for word in lemmas if word not in stopwords])

#stop_words_filename = 'D:/GooDrive/0_2018_Edu_CL2/0_2018_Edu_HSE_Bak_CL2/WSD/WSD_Classifier/stoplist_russian.txt' 	# одно слово на строке
stop_words = load_stop_words(stopw)
raw_texts = ['Он', 'еле', 'дождался', 'пока', 'я', 'доковыряюсь', 'ключом', 'в', 'замке', 'и', 'не', 'снимая']
print(remove_stop_words(raw_texts, stop_words))

Он еле дождался доковыряюсь ключом замке снимая


### Найти наиболее частотные слова в корпусе (for reduced vectors)

In [513]:
import string
from collections import Counter

f1 = open('context_words.txt', 'r', encoding = 'utf-8')
text = f1.read()
text = text.strip('\n')

exclude = set(string.punctuation)
text = ''.join(ch for ch in text if ch not in exclude)
text = text.split(' ')

# remove stop-words
text = remove_stop_words(text, stop_words)

# lemmatize
text = lemmatize(text)

# counter
counts = Counter(text)
counts = sorted(counts.items(), key=lambda pair: pair[1], reverse=True)

# list of most popular words
popular = []

for i in counts:
    if i[1] > 10:
        popular.append(i[0])

print(len(popular))

205


### 1. Загрузка и адаптация корпуса (initial vectors)



In [514]:
import codecs

def remove_stop_words(lemmas, stopwords):
    return ' '.join([word for word in lemmas if word not in stopwords])

def load_files(filename):
    # список, где будут накапливаться результаты
    results = []
    
    # открываем файл как переменную inp_file.
    # Важно помнить, что в этой переменной хранится не сам текстовый файл, а своего рода указатель на него.
    # Файл еще предстоит прочитать.
    # Конструкция with следит за тем, чтобы переменная, которая после as, существовала только внутри блока.
    # Благодаря этому файл закроется сразу же после использования
    with(codecs.open(filename, encoding = 'utf-8')) as inp_file:
        # читаем первую строку "в никуда" (не сохраняем результат).
        # Первая строка у нас — это названия колонок, они нам не нужны
        inp_file.readline()
        # Для каждой строки в файле
        for line in inp_file:
            # откусываем концы строк
            line = line.strip('\r\n')
            # строка.count(подстрока) возвращает количество вхождений подстроки в строку
            if line.count('\t') != 1:
            # специальное ключевое слово для перехода к следующей итерации цикла
                continue
            # делим строку по табуляции и складываем в каждую из двух переменных очередное значение
            sence, context = line.split('\t')
            # добавляем кортеж (sence, context) в общий список результатов
            
            cx = lemmatize(context)
            #cx = context
            cx = remove_stop_words(cx, stop_words)
            results.append((int(sence), cx))
    # возвращаем результаты в виде списка кортежей (sence, context)
    return results

### 1. Загрузка и адаптация корпуса для уменьшенной размерности вектора (reduced vectors)

In [460]:
import codecs

def remove_stop_words(lemmas, stopwords): # also removes popular words
    return ' '.join([word for word in lemmas if word not in stopwords and word not in popular])

def load_files(filename):
    # список, где будут накапливаться результаты
    results = []
    
    with(codecs.open(filename, encoding = 'utf-8')) as inp_file:
        inp_file.readline()
        for line in inp_file:
            line = line.strip('\r\n')
            if line.count('\t') != 1:
                continue
            sence, context = line.split('\t')  
            cx = lemmatize(context)
            # remove most popular words + stop words
            cx = remove_stop_words(cx, stop_words)
                    
            results.append((int(sence), cx))

    return results

In [515]:
# указываем абсолютный или относительный путь
data = load_files('castle-lock.txt')

In [516]:
with codecs.open('lemmatized_data.txt', mode='w', encoding='utf-8') as wr:
    wr.write('sense\tcontext(lemmatized)\r\n')
    for item in data:
        wr.write(str(item[0]) + '\t' + item[1] + "\r\n")

1. Перевод корпуса в векторы признаков, feature vectors

In [517]:
# математика
import numpy

# разные классификаторы
from sklearn.naive_bayes import MultinomialNB       # Multinomial Naive Bayes
#from sklearn import tree
from sklearn.tree import DecisionTreeClassifier     # Decision Tree
from sklearn.ensemble import RandomForestClassifier # Random Forest
#from sklearn.svm import LinearSVC

# векторизация
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# деление корпуса train/test
from sklearn.model_selection import cross_val_score # cross-validation
from sklearn.model_selection import train_test_split

# Pipeline
from sklearn.pipeline import Pipeline

In [518]:
x_context = [item[1] for item in data]
y_labels = [item[0] for item in data]
print(x_context[:5])
print(y_labels[1])

['горный склон конный спорт рыбалка круиз исторический место пляжный', 'принцесса башня смело спасать распрекрасный за уносить жить', 'например сыграть роль огонь сказочный или светлячок летний сад для модный', 'из семь чудо свет величественный святой петр xv сегодня замок располагать', 'замок святой петр xv сегодня располагать музей подводный археология']
1


In [519]:
def tokenize(input):
    return input.split(' ')

In [520]:
#classifiers = [('SVM', LinearSVC), ('Bayes', MultinomialNB)]
classifiers = [('Bayes', MultinomialNB), ('Decision Tree', DecisionTreeClassifier), ('Random Forest', RandomForestClassifier)]
vectorizers = [('Counts', CountVectorizer), ('TfIdf', TfidfVectorizer)]

### F-score for classifier

In [521]:
def score_classifier(clf, metric='f1'):
    scores = cross_val_score(pipeline, numpy.asarray(x_context), numpy.asarray(y_labels), cv=5, scoring=metric)
    score = sum(scores) / len(scores)    
    return score

### Pipeline score estimation

In [522]:
for clf in classifiers:
    for vctr in vectorizers:
        pipeline = Pipeline([
    ('vectorizer', vctr[1]()),
    ('classifier', clf[1]()) ])
        print (clf[0], vctr[0], score_classifier(pipeline))

Bayes Counts 0.89708488045
Bayes TfIdf 0.889761477254
Decision Tree Counts 0.812041803886
Decision Tree TfIdf 0.78332810719
Random Forest Counts 0.840131155158
Random Forest TfIdf 0.811500298455


In [523]:
#import sklearn

from nltk.corpus import stopwords
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold
from sklearn.naive_bayes import MultinomialNB
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.model_selection import train_test_split

import numpy as np

#from sklearn.feature_extraction.text import BinaryVectorizer, CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import os, sys, codecs

In [524]:
texts = [item[1] for item in data]
for i in range(10):
    print (texts[i])
    
len(texts)

print (3974 *16344 * 4)

горный склон конный спорт рыбалка круиз исторический место пляжный
принцесса башня смело спасать распрекрасный за уносить жить
например сыграть роль огонь сказочный или светлячок летний сад для модный
из семь чудо свет величественный святой петр xv сегодня замок располагать
замок святой петр xv сегодня располагать музей подводный археология
прибор вешать большой зал дворец масштабный фамильный имение совместно
размер первоначальный выборгский основывать семь рано же
информация о товар конструктор рыцарский 2700 руб фантастический набор деревянный
другой можно возводить настоящий рыцарский упаковка или придумывать игра
например находиться руины девичий 110 километр от измир располагать
259804224


### Vectorization: Tf idf

In [525]:
vectorizer = TfidfVectorizer() # Tf idf

texts = [item[1] for item in data]
vectorized = vectorizer.fit_transform(texts)
vectorized

print (vectorized[0], vectorized[0].shape)
print('\n')

names = vectorizer.get_feature_names()

  (0, 828)	0.30657838977
  (0, 3259)	0.35877136719
  (0, 1543)	0.35877136719
  (0, 3418)	0.35877136719
  (0, 3125)	0.35877136719
  (0, 1626)	0.35877136719
  (0, 1366)	0.292911162426
  (0, 1815)	0.218886679261
  (0, 2499)	0.35877136719 (1, 4149)




#### Classifier: Random Forest

In [526]:
clf = RandomForestClassifier(n_estimators=250)

In [527]:
y_labels = [item[0] for item in data]
X_train, X_test, y_train, y_test = train_test_split(vectorized, y_labels, test_size=0.1, random_state=42)

clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=250, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [528]:
items = clf.predict(X_test.toarray())
right = 0
for i in range(len(items)):
    if items[i] == y_test[i]:
        right += 1
        
print ('Total: %d\nRight: %d\nScore: %.3f' % (len(items), right, float(right) / len(items)))

words = {}
for i in range(len(clf.feature_importances_)):
    words[names[i]] = clf.feature_importances_[i]
    
#for item in sorted(words, key = lambda word: words[word], reverse=True):
#    print (item, words[item])

Total: 140
Right: 124
Score: 0.886


#### Tf idf & Random forest f-score:

In [529]:
from sklearn.metrics import f1_score

f1_score(y_test, items, labels=None, pos_label=1, average='binary', sample_weight=None)

0.90361445783132521

#### Classifier: Multinomial naive Bayes

In [530]:
clf = MultinomialNB()

In [531]:
y_labels = [item[0] for item in data]
X_train, X_test, y_train, y_test = train_test_split(vectorized, y_labels, test_size=0.1, random_state=42)
clf.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [532]:
items = clf.predict(X_test.toarray())
right = 0
for i in range(len(items)):
    if items[i] == y_test[i]:
        right += 1
        
print ('Total: %d\nRight: %d\nScore: %.3f' % (len(items), right, float(right) / len(items)))

#words = {}
#for i in range(len(clf.feature_importances_)):
#    words[names[i]] = clf.feature_importances_[i]     # no feature_importances_ in MultinomialNB
    
#for item in sorted(words, key = lambda word: words[word], reverse=True):
#    print (item, words[item])

Total: 140
Right: 124
Score: 0.886


#### Tf idf & multinomial naive Bayes f-score:

In [533]:
f1_score(y_test, items, labels=None, pos_label=1, average='binary', sample_weight=None)

0.8904109589041096

#### Classifier: Decision Tree

In [534]:
clf = tree.DecisionTreeClassifier()

In [535]:
y_labels = [item[0] for item in data]
X_train, X_test, y_train, y_test = train_test_split(vectorized, y_labels, test_size=0.1, random_state=42)
clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [536]:
items = clf.predict(X_test.toarray())
right = 0
for i in range(len(items)):
    if items[i] == y_test[i]:
        right += 1
        
print ('Total: %d\nRight: %d\nScore: %.3f' % (len(items), right, float(right) / len(items)))

words = {}
for i in range(len(clf.feature_importances_)):
    words[names[i]] = clf.feature_importances_[i]
    
#for item in sorted(words, key = lambda word: words[word], reverse=True):
#    print (item, words[item])

Total: 140
Right: 111
Score: 0.793


#### Tf idf & Decision Tree f-score:

In [537]:
f1_score(y_test, items, labels=None, pos_label=1, average='binary', sample_weight=None)

0.81987577639751552

### Vectorization: Count 

In [538]:
vectorizer = CountVectorizer() # Count

texts = [item[1] for item in data]
vectorized = vectorizer.fit_transform(texts)
vectorized

print (vectorized[0], vectorized[0].shape)

  (0, 2499)	1
  (0, 1815)	1
  (0, 1366)	1
  (0, 1626)	1
  (0, 3125)	1
  (0, 3418)	1
  (0, 1543)	1
  (0, 3259)	1
  (0, 828)	1 (1, 4149)


#### Classifier: Random Forest

In [539]:
clf = RandomForestClassifier(n_estimators=250)

In [540]:
y_labels = [item[0] for item in data]
X_train, X_test, y_train, y_test = train_test_split(vectorized, y_labels, test_size=0.1, random_state=42)
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=250, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [541]:
items = clf.predict(X_test.toarray())
right = 0
for i in range(len(items)):
    if items[i] == y_test[i]:
        right += 1
    #print items[i], y_test[i]
names = vectorizer.get_feature_names()
words = {}
print ('Total: %d\nRight: %d\nScore: %.3f' % (len(items), right, float(right) / len(items)))
for i in range(len(clf.feature_importances_)):
    words[names[i]] = clf.feature_importances_[i]
    
#for item in sorted(words, key = lambda word: words[word], reverse=True):
#    print (item, words[item])

Total: 140
Right: 119
Score: 0.850


#### Count & Random Forest f-score:

In [542]:
f1_score(y_test, items, labels=None, pos_label=1, average='binary', sample_weight=None)

0.87272727272727268

#### Classifier: Multinomial naive Bayes

In [543]:
clf = MultinomialNB()

In [544]:
y_labels = [item[0] for item in data]
X_train, X_test, y_train, y_test = train_test_split(vectorized, y_labels, test_size=0.1, random_state=42)
clf.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [545]:
items = clf.predict(X_test.toarray())
right = 0
for i in range(len(items)):
    if items[i] == y_test[i]:
        right += 1
        
print ('Total: %d\nRight: %d\nScore: %.3f' % (len(items), right, float(right) / len(items)))

#words = {}
#for i in range(len(clf.feature_importances_)):
#    words[names[i]] = clf.feature_importances_[i]     # no feature_importances_ in MultinomialNB
    
#for item in sorted(words, key = lambda word: words[word], reverse=True):
#    print (item, words[item])

Total: 140
Right: 123
Score: 0.879


#### Count & Multinomial naive Bayes f-score:

In [546]:
f1_score(y_test, items, labels=None, pos_label=1, average='binary', sample_weight=None)

0.88435374149659862

#### Classifier: Decision Tree

In [547]:
clf = tree.DecisionTreeClassifier()

In [548]:
y_labels = [item[0] for item in data]
X_train, X_test, y_train, y_test = train_test_split(vectorized, y_labels, test_size=0.1, random_state=42)
clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [549]:
items = clf.predict(X_test.toarray())
right = 0
for i in range(len(items)):
    if items[i] == y_test[i]:
        right += 1
        
print ('Total: %d\nRight: %d\nScore: %.3f' % (len(items), right, float(right) / len(items)))

words = {}
for i in range(len(clf.feature_importances_)):
    words[names[i]] = clf.feature_importances_[i]
    
#for item in sorted(words, key = lambda word: words[word], reverse=True):
#    print (item, words[item])

Total: 140
Right: 110
Score: 0.786


#### Count & Decision Tree f-score:

In [550]:
f1_score(y_test, items, labels=None, pos_label=1, average='binary', sample_weight=None)

0.8125

## Testing 10 extra contexts (Initial vectors + Tf idf + Random Forest)

In [551]:
vectorizer = TfidfVectorizer() # Tf idf

test_data = load_files('test_contexts.txt') # includes 10 extra contexts
test_data1 = [item[1] for item in test_data]

vectorized = vectorizer.fit_transform(test_data1)
print(vectorized.shape[0])

v = vectorized.toarray()

v1 = v[:1400]      # for training
v2 = v[1400:1410]  # 10 test sentences

clf = RandomForestClassifier(n_estimators=250)
y_labels = [item[0] for item in test_data[:1400]]

y_labels = [item[0] for item in test_data[:1400]]
X_train, X_test, y_train, y_test = train_test_split(v1, y_labels, test_size=0.1, random_state=42)

clf.fit(X_train, y_train)


1410


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=250, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [552]:
items = clf.predict(X_test)
right = 0
for i in range(len(items)):
    if items[i] == y_test[i]:
        right += 1
        
print ('Total: %d\nRight: %d\nScore: %.3f' % (len(items), right, float(right) / len(items)))

Total: 140
Right: 123
Score: 0.879


In [553]:
f1_score(y_test, items, labels=None, pos_label=1, average='binary', sample_weight=None)

0.89820359281437123

In [554]:
test_items = clf.predict(v2)
for i in test_items:
    print(i)

1
0
1
1
1
0
0
0
0
1


* Первые 5 примеров -- 1 (castle), последние 5 -- 0 (lock)
* <b>Результат: 8/10</b> (ошибочная классификация примеров 2 и 10):
* 2 -- 'ночью замок был освещен и был открыт для посетителей и ночных экскурсий'
* 10 -- 'ребенок пытался вставить карандаш в скважину замка и громко орал'